In [2]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

pw = '27111998'
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [3]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.execute(query)
        else:
            cursor.execute(query, varlist)
        connection.commit()
        return cursor
    except Error as err:
        print(f"Error: '{err}'")
        return cursor

def executemany_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.executemany(query)
        else:
            cursor.executemany(query, varlist)
        connection.commit()
        return cursor
    except Error as err:
        print(f"Error: '{err}'")
        return cursor

In [4]:
db = 'crypto_db'
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [5]:
data = pd.read_csv('logreg_model_not_sorted.csv')
considered_market = data['market']
considered_market

0       1INCHBTC
1      1INCHUSDT
2       AAVEUSDT
3         ACABTC
4        ACAUSDT
         ...    
601      ZENUSDT
602       ZILBTC
603      ZILUSDT
604       ZRXBTC
605      ZRXUSDT
Name: market, Length: 606, dtype: object

In [5]:
data.iloc[0,0:2]

Unnamed: 0           0
market        1INCHBTC
Name: 0, dtype: object

In [11]:
arr_max_price = np.zeros(len(considered_market))
arr_cur_price = np.zeros(len(considered_market))
arr_init_price = np.zeros(len(considered_market))
for i in range(len(considered_market)):
    try:
        market = considered_market[i]
        market = market.replace('/', '')
        print(market)
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        cur = execute_query(connection,query)
        res = cur.fetchall()
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        df_temp = df.loc[(df['timestamp'] > 1708214400000) & (df['timestamp'] <= 1709856000000)]
        max_price = max(df_temp['close'])
        last_price = df_temp['close'].iloc[-1]
        init_price = df_temp['close'].iloc[0]
        arr_max_price[i] = max_price
        arr_cur_price[i] = last_price
        arr_init_price[i] = init_price
    except Exception as err:
        print(err)

1INCHBTC
1INCHUSDT
AAVEUSDT
ACABTC
ACAUSDT
ACEBTC
ACEUSDT
ACHBTC
ACHUSDT
ACMUSDT
ADABTC
ADAUSDT
ADXBTC
ADXUSDT
AERGOBTC
AERGOUSDT
AGIXBTC
AGIXUSDT
AGLDBTC
AGLDUSDT
AIBTC
AIUSDT
AKROUSDT
ALCXBTC
ALCXUSDT
ALGOBTC
ALGOUSDT
ALICEBTC
ALICEUSDT
ALPACABTC
ALPACAUSDT
ALPHABTC
ALPHAUSDT
ALPINEBTC
ALPINEUSDT
ALTBTC
ALTUSDT
AMPUSDT
ANKRBTC
ANKRUSDT
APEBTC
APEUSDT
API3BTC
API3USDT
APTBTC
APTUSDT
ARBBTC
ARBUSDT
ARKMBTC
ARKMUSDT
ARPABTC
ARPAUSDT
ARBTC
ARUSDT
ASRUSDT
ASTRUSDT
ASTBTC
ATABTC
ATAUSDT
ATMUSDT
ATOMBTC
ATOMUSDT
AUCTIONBTC
AUCTIONUSDT
AUDIOBTC
AUDIOUSDT
AVAXBTC
AVAXUSDT
AVABTC
AVAUSDT
AXSBTC
AXSUSDT
BADGERBTC
BADGERUSDT
BALBTC
BALUSDT
BANDBTC
BANDUSDT
BARUSDT
BATBTC
BATUSDT
BCHBTC
BCHUSDT
BEAMXUSDT
BELBTC
BELUSDT
BETABTC
BETAUSDT
BICOBTC
BICOUSDT
BIFIUSDT
BLURBTC
BLURUSDT
BLZBTC
BLZUSDT
BNBDOWNUSDT
max() arg is an empty sequence
BNBUPUSDT
max() arg is an empty sequence
BNBBTC
BNBUSDT
BNTBTC
BNTUSDT
BNXBTC
BNXUSDT
BONDBTC
BONDUSDT
BONKUSDT
BSWUSDT
BTCDOWNUSDT
max() arg is an empty sequence
B

In [12]:
price_data = pd.DataFrame({'market': data['market'], 'init_price': arr_init_price, 'max_price': arr_max_price, 'cur_price': arr_cur_price})
price_data.to_csv('current_price.csv')

In [7]:
market = considered_market[10]
market = market.replace('/', '')
print(market)
query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
cur = execute_query(connection,query)
res = cur.fetchall()
df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
df

ADABTC


,timestamp,open,high,low,close,volume
0,1512043200000,0.000009,0.000089,0.000009,0.000012,5468800.0
1,1512046800000,0.000012,0.000012,0.000011,0.000011,4706220.0
2,1512050400000,0.000011,0.000013,0.000011,0.000012,7824210.0
3,1512054000000,0.000012,0.000012,0.000011,0.000012,6190480.0
4,1512057600000,0.000012,0.000013,0.000012,0.000012,8185850.0
...,...,...,...,...,...,...
54431,1708239600000,0.000012,0.000012,0.000012,0.000012,808819.0
54432,1708243200000,0.000012,0.000012,0.000012,0.000012,1186110.0
54433,1708246800000,0.000012,0.000012,0.000012,0.000012,1292330.0
54434,1708250400000,0.000012,0.000012,0.000012,0.000012,200110.0


In [8]:
from datetime import datetime

last_update = datetime.fromtimestamp(df['timestamp'].iloc[-1]/1000)
last_update

datetime.datetime(2024, 2, 18, 17, 0)

In [9]:
inittime = datetime(1971, 1, 2, 7, 0, 0)
datetime.timestamp(np.nan)

TypeError: descriptor 'timestamp' for 'datetime.datetime' objects doesn't apply to a 'float' object

In [13]:
datetime.fromtimestamp(1708257600000/1000)

datetime.datetime(2024, 2, 18, 19, 0)

In [3]:
print('tes'.upper())

TES
